In [1]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

sess = sagemaker.Session()
role = "arn:aws:iam::123456789012:role/SageMakerExecutionRole"

huggingface_model = HuggingFaceModel(
    model_data="s3://your-bucket/distilgpt2-sdsu.tar.gz",
    role=role,
    transformers_version="4.26.0",
    pytorch_version="1.13.1",
    py_version="py39",
    entry_point="inference.py"  # defines model_fn and predict_fn
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="AdmissionsAgentEndpoint"
)

ModuleNotFoundError: No module named 'sagemaker'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def model_fn(model_dir):
    model = AutoModelForCausalLM.from_pretrained(model_dir)
    tokenizer = AutoTokenizer.from_pretrained(model_dir)
    return {"model": model, "tokenizer": tokenizer}

def predict_fn(data, model_artifacts):
    input_text = data["inputs"]
    inputs = model_artifacts["tokenizer"](input_text, return_tensors="pt")
    outputs = model_artifacts["model"].generate(**inputs, max_new_tokens=50)
    return model_artifacts["tokenizer"].decode(outputs[0])
